In [1]:
from env import ImgRouterEvalEnv
from lightning_lite.utilities.seed import seed_everything
from bandit_algo import BanditAlgorithm
from preference_estimator import (
    EmpiricalEstimator,
    NeuralEstimator,
    RecurrentNeuralEstimator,
    RecurrentNeuralEstimatorV0,
)
from exploration_strategy import (
    PerArmExploration,
    UniformEpsilonExploration,
)
import numpy as np

In [2]:
num_candidates = 64
cfg = {
    "num_candidates": num_candidates,
}
env = ImgRouterEvalEnv(cfg)

In [3]:
seed_everything(0)

obs = env.reset()
obs

Global seed set to 0


MNIST num parameters: 14648


array([[1, 9],
       [1, 3],
       [1, 9],
       [1, 5],
       [0, 7],
       [1, 6],
       [1, 4],
       [0, 8],
       [1, 4],
       [0, 4],
       [1, 1],
       [1, 0],
       [0, 7],
       [0, 7],
       [0, 4],
       [0, 6],
       [0, 3],
       [0, 4],
       [0, 2],
       [0, 5],
       [1, 3],
       [1, 3],
       [0, 3],
       [1, 4],
       [1, 5],
       [0, 1],
       [0, 1],
       [1, 4],
       [0, 2],
       [1, 8],
       [0, 9],
       [0, 3],
       [1, 2],
       [0, 1],
       [0, 0],
       [0, 7],
       [1, 0],
       [1, 4],
       [0, 2],
       [1, 6],
       [0, 7],
       [1, 3],
       [0, 7],
       [0, 6],
       [1, 4],
       [1, 5],
       [0, 8],
       [0, 7],
       [1, 4],
       [1, 1],
       [0, 0],
       [1, 4],
       [1, 5],
       [1, 7],
       [1, 9],
       [0, 3],
       [1, 3],
       [1, 8],
       [0, 7],
       [0, 0],
       [1, 5],
       [0, 8],
       [1, 5],
       [1, 7]])

In [4]:
batch_x, batch_z, batch_y = env._get_full_obs()
batch_y

tensor([19, 13, 19, 15,  7, 16, 14,  8, 14,  4, 11, 10,  7,  7,  4,  6,  3,  4,
         2,  5, 13, 13,  3, 14, 15,  1,  1, 14,  2, 18,  9,  3, 12,  1,  0,  7,
        10, 14,  2, 16,  7, 13,  7,  6, 14, 15,  8,  7, 14, 11,  0, 14, 15, 17,
        19,  3, 13, 18,  7,  0, 15,  8, 15, 17])

In [5]:
num_tasks = 2
num_cls = 10
reduce_fator = 2
num_slates = num_candidates // reduce_fator
estimator = EmpiricalEstimator(num_tasks, num_cls)
# estimator = RecurrentNeuralEstimatorV0(num_tasks, num_cls)
# explore = PerArmExploration(num_tasks, num_cls, num_slates)
explore = UniformEpsilonExploration(num_tasks, num_cls, num_slates)
algo = BanditAlgorithm(
        estimator,
        explore,
    )

In [6]:
obs = env.reset()
# print("initial obs:", obs)
done = False
step_rewards = []
pred_losses = []
step = 0
while not done:
    action = algo.predict(obs)
    next_obs, reward, done, info = env.step(action)
    algo.update_estimator(obs, action, info["rewards"])
    obs = next_obs
    step_rewards.append(reward)
    pred_loss = (algo.estimator.Q - env.Q) ** 2
    pred_losses.append(pred_loss)
    step += 1

print("current samples after training:")
print(env.current_samples)
# routed_data = env.current_samples - existing_samples
routed_data = env.current_samples
print("routed data:")
print(routed_data)
print(np.mean(step_rewards))

MNIST num parameters: 14648
Q_values [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
step 0 epsilon 2.0
routed_batch_z (tensor([0, 1]), tensor([16, 16]))
uniform: 0.003117799758911133 -0.0024323463439941406
delta:
[[ 0.          0.0093534   0.0093534   0.0062356   0.0031178   0.0031178
   0.0031178   0.0062356   0.0062356   0.0031178 ]
 [-0.00486469 -0.00729704 -0.00243235  0.         -0.00972939 -0.00243235
  -0.00243235 -0.00486469 -0.00243235 -0.00243235]]


AttributeError: 'ImgRouterEvalEnv' object has no attribute 'target_samples'